In [1]:
#Import libraries
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [2]:
#Load brands data
brands = []
for line in open('LOCATION/brands.json', 'r'):
    brands.append(json.loads(line))
df_brands = pd.DataFrame.from_dict(brands,orient='columns')
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827


In [3]:
#Check for original cpg column
a = set([])
for i in range(df_brands.shape[0]):
    a.add(df_brands.loc[i,'cpg']['$ref'])
a

{'Cogs', 'Cpgs'}

In [4]:
#Clean brands data
ids=[]
cpg_ids = []
cpg_refs = []
for i in range(df_brands.shape[0]):
    ids.append(df_brands.iloc[i,0]['$oid'])
    #separating original cpg column in to two separate columns
    cpg_ids.append(df_brands.iloc[i,4]['$id']['$oid'])
    cpg_refs.append(df_brands.iloc[i,4]['$ref'])
#Creating columns with clean data
df_brands['id'] = ids
df_brands['cpg_id'] = cpg_ids
df_brands['cpg_ref'] = cpg_refs
#Dropping columns with raw data
df_brands.drop(labels=['_id','cpg'],axis=1, inplace=True)
df_brands = df_brands.reindex(columns=['id','barcode','category','categoryCode','cpg_id','cpg_ref','name','brandCode'])
df_brands.head()

,id,barcode,category,categoryCode,cpg_id,cpg_ref,name,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,TEST BRANDCODE @1612366146827


In [5]:
#Creating category table and dropping categoryCode column from brands table
df_category = df_brands[['category','categoryCode']]
df_category = df_category.drop_duplicates(ignore_index=True)
df_category.dropna(how='all',inplace=True)
df_category.reset_index(inplace=True)
df_brands.drop(columns=['categoryCode'],inplace=True)
df_category

,index,category,categoryCode
0,0,Baking,BAKING
1,1,Beverages,BEVERAGES
2,2,Candy & Sweets,CANDY_AND_SWEETS
3,3,Condiments & Sauces,NaN
4,4,Canned Goods & Soups,NaN
5,5,Baking,NaN
6,7,Magazines,NaN
7,8,Breakfast & Cereal,NaN
8,9,Beer Wine Spirits,NaN
9,10,Health & Wellness,HEALTHY_AND_WELLNESS


In [6]:
#Load receipts data
receipts = []
for line in open('LOCATION/receipts.json', 'r'):
    receipts.append(json.loads(line))
df_receipts = pd.DataFrame.from_dict(receipts,orient='columns')
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [8]:
#datetime check
dt = datetime.fromtimestamp(df_receipts.loc[0,'dateScanned']['$date']/1000)
print(dt)

2021-01-03 10:25:31


In [9]:
#Clean receipts data
ids=[]
lastlogins = []
createddates = []
scanneddates = []
finisheddates = []
modifydates = []
pointsawardeddates = []
purchasedates = []
for i in range(df_receipts.shape[0]):
    ids.append(df_receipts.iloc[i,0]['$oid'])
    createddates.append(datetime.fromtimestamp(df_receipts.iloc[i,3]['$date']/1000))
    scanneddates.append(datetime.fromtimestamp(df_receipts.iloc[i,4]['$date']/1000))
    #Using try and except to consider the case of null values
    try:
        finisheddates.append(datetime.fromtimestamp(df_receipts.iloc[i,5]['$date']/1000))
    except:
        finisheddates.append(np.nan)
    modifydates.append(datetime.fromtimestamp(df_receipts.iloc[i,6]['$date']/1000))
    try:
        pointsawardeddates.append(datetime.fromtimestamp(df_receipts.iloc[i,7]['$date']/1000))
    except:
        pointsawardeddates.append(np.nan)
    try:
        purchasedates.append(datetime.fromtimestamp(df_receipts.iloc[i,9]['$date']/1000))
    except:
        purchasedates.append(np.nan)
    

#Creating columns with clean data
df_receipts['id'] = ids
df_receipts['createdDate_clean'] = createddates
df_receipts['dateScanned_clean'] = scanneddates
df_receipts['finishedDate_clean'] = finisheddates
df_receipts['modifyDate_clean'] = modifydates
df_receipts['pointsAwardedDate_clean'] = pointsawardeddates
df_receipts['purchaseDate_clean'] = purchasedates
#Dropping columns with raw data
df_receipts.drop(labels=['createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate'],axis=1, inplace=True)
#Renaming new columns to original names
df_receipts.rename(columns={'createdDate_clean':'createDate', 'dateScanned_clean':'dateScanned','finishedDate_clean':'finishedDate','modifyDate_clean':'modifyDate','pointsAwardedDate_clean':'pointsAwardedDate','purchaseDate_clean':'purchaseDate'},inplace=True)
df_receipts = df_receipts.reindex(columns=['id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent',
       'userId'])
df_receipts.head()

,id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,NaT,2021-01-03 10:25:42,NaT,5,2021-01-02 19:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [10]:
#Exploring original rewardsReceiptItemList column
df_receipts.loc[df_receipts['rewardsReceiptItemList'].isna(),'rewardsReceiptItemList'].unique
df_receipts.loc[93,'rewardsReceiptItemList']
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    object        
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptItemList   679 non-null    object        
 12  rewardsReceiptStatus     1119 non-

In [11]:
#Creating a separate receipt_items table
df_receipt_items = pd.DataFrame()
w=0
for i in range(df_receipts.shape[0]):
    try:
        #create separate rows for each item in order
        for j in range(len(df_receipts.loc[i,'rewardsReceiptItemList'])):
            df_receipt_items.loc[w,'id'] = df_receipts.loc[i,'id']
            for k in df_receipts.loc[i,'rewardsReceiptItemList'][j].keys():
                #create fields and add values to the table
                if k in df_receipt_items.columns:
                    df_receipt_items.loc[w,k] = df_receipts.loc[i,'rewardsReceiptItemList'][j][k]
                
                else:
                    df_receipt_items.loc[w,k] = df_receipts.loc[i,'rewardsReceiptItemList'][j][k]
            w=w+1
    except:
        continue    
df_receipts.drop(labels=['rewardsReceiptItemList'],axis=1, inplace=True)        
df_receipt_items.head()

,id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Load users data
users = []
for line in open('LOCATION/users.json', 'r'):
    users.append(json.loads(line))
df_users = pd.DataFrame.from_dict(users,orient='columns')
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [13]:
#Clean users data
ids=[]
lastlogins = []
createddates = []
for i in range(df_users.shape[0]):
    ids.append(df_users.iloc[i,0]['$oid'])
    #Using try and except to consider the case of null values
    try:
        lastlogins.append(datetime.fromtimestamp(df_users.iloc[i,3]['$date']/1000))
    except:
        lastlogins.append(np.nan)
    createddates.append(datetime.fromtimestamp(df_users.iloc[i,2]['$date']/1000))
        
#Creating columns with clean data
df_users['id'] = ids
df_users['createdDate_clean'] = createddates
df_users['lastLogin_clean'] = lastlogins
#Dropping columns with raw data
df_users.drop(labels=['createdDate','_id','lastLogin'],axis=1, inplace=True)
#Renaming new columns to original names
df_users.rename(columns={'createdDate_clean':'createdDate', 'lastLogin_clean':'lastLogin'},inplace=True)
df_users = df_users.reindex(columns=['id','active','createdDate','lastLogin','role','signUpSource','state'])
df_users.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 10:25:30.554,2021-01-03 10:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858,consumer,Email,WI


In [14]:
#saving csv files for each table

df_users.to_csv('users.csv',index=False)
df_brands.to_csv('brands.csv',index=False)
df_receipt_items.to_csv('receipt_item.csv',index=False)
df_receipts.to_csv('receipts.csv',index=False)
df_category.to_csv('category.csv',index=False)